<a href="https://colab.research.google.com/github/CarlottaVattolo/Esame/blob/main/Studenti_IntroToORTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install an Import OR-Tools Package

In [ ]:
!pip install ortools

Import LP solver wrapper 

In [ ]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np

### Solve the Linear Problem
\begin{align}
  \text{maximize}\ &250x_1 + 230x_2 + 110x_3 + 350x_4 + 110x_5\\
                  &s.t.\\
                  &0\le\ x_i \lt \infty\\
                  &2x_1 + 1.5x_2 + 0.5x_3 + 2.5x_4 \leq 100.\\
                  &0.5x_1 + 0.25x_2 + 0.25x_3 + x_4 \leq 50\\
                  &x_1 + x_2 \geq 10\\
                  &x_3 = x_5\\
\end{align}

Create the solver using the GLOP backend, documentation @ https://developers.google.com/optimization/reference/python/linear_solver/pywraplp

In [ ]:
solver = pywraplp.Solver.CreateSolver('GLOP')

Create variables $x_1, \dots, x_4$ in the range $0\le x_i \lt \infty$

In [ ]:
lb = 0
ub = float('inf')
x = np.array([solver.NumVar(lb, ub, f"x_{i}") for i in range(1, 6)])

Access variables in the solver

In [ ]:
print(f"Number of variables {solver.NumVariables()}")
x_1_ptr, x_2_ptr = solver.variable(0), solver.variable(1)
print("Type of variables: ", type(x_1_ptr), type(x_1_ptr))

Add constraints 
\begin{align}
0 \leq\ &2x_1 + 1.5x_2 + 0.5x_3 + 2.5x_4 \leq 100.\\
0 \leq\ &0.5x_1 + 0.25x_2 + 0.25x_3 + x_4 \leq 50
\end{align}

In [ ]:
ct1_lb = 0
ct1_ub = 100
ct1 = solver.Constraint(ct1_lb, ct1_ub, "ct1")

In [ ]:
print(f"Number of constraints: {solver.NumConstraints()}")
print(f"Constraint lower and upper bounds: {ct1.lb()}, {ct1.ub()}")
print(f"Default variable coefficients in the constraint: {[ct1.GetCoefficient(xi) for xi in x]}")
ct1.SetCoefficient(x[0], 2.)
ct1.SetCoefficient(x[1], 1.5)
ct1.SetCoefficient(x[2], 0.5)
ct1.SetCoefficient(x[3], 2.5)
ct1.SetCoefficient(x[4], 0.7)
print(f"Set variable coefficients in the constraint: {[ct1.GetCoefficient(xi) for xi in x]}")

In [ ]:
ct2_lb = 0
ct2_ub = 50
ct2_coefs = np.array([0.5, 0.25, 0.25, 1, 0.3])
x_arr = np.array(x)
ct2 = solver.Add(ct2_lb <= sum(ct2_coefs * x_arr) <= ct2_ub)
print(f"Constraint 2 coefficients: {[ct2.GetCoefficient(xi) for xi in x]}")



Add constraints 
\begin{align}
x_1 + x_2 &\geq 10.\\
x_3 &= x_5 
\end{align}

In [ ]:
ct3 = solver.Add(x[0] + x[1] >= 10)
ct4 = solver.Add(x[2] == x[4])

Create the objective function $250x_1 + 230x_2 + 110x_3 + 350x_4 +110x_5$ and set it for maximization

In [ ]:
obj = solver.Objective()
obj.SetMaximization()
print(f"Default variable coefficients in the objective: {[obj.GetCoefficient(xi) for xi in x]}")
obj_coefs = [250, 230, 110, 350, 110]
for xi, ci in zip(x, obj_coefs):
  obj.SetCoefficient(xi, ci)
print(f"Set variable coefficients in the objective:  {[obj.GetCoefficient(xi) for xi in x]}")

# Or more concice:
#obj_coefs = np.array([250, 230, 110, 350])
#solver.Maximize(sum(x * obj_coefs))

Solve the optimization problem

In [ ]:
status = solver.Solve()
print('Solution:')
print('Objective value =', solver.Objective().Value())
for xi in x:
  print(f"{xi} = {xi.solution_value()}")

### Solve the dual problem

In [ ]:
dual_solver = ...

Add variables 

In [ ]:
p = ...
p_arr = np.array([...])

Add constraints 

In [ ]:
dual_constraints = ...

Set objective function 

In [ ]:
...

In [ ]:
dual_solver.Solve()
print('Solution:')
print('Objective value =', dual_solver.Objective().Value())
for pi in dual_solver.variables():
  print(f"{pi} = {pi.solution_value()}")

Check that solutions match with dual values provided by OR tools for each constraint in the primal and dual

In [ ]:
print("Primal constraints dual values:")
print(f"\tct  dual value = {ct1.dual_value():.2f}")
print(f"\tct2 dual value = {ct2.dual_value():.2f}")
print(f"\tct3 dual value = {ct3.dual_value():.2f}")
print(f"\tct4 dual value = {ct4.dual_value():.2f}")

In [ ]:
print("Dual constraints dual values:")
for i, c in enumerate(dual_constraints):
  print(f"\t ct{i} dual value: {c.dual_value():.2f}")